In [1]:
import pandas as pd
import keras
from keras.models import load_model
import numpy as np
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
model_path = './models/basic_c_and_n_regression_hydrophobic_only.h5'

df = pd.read_csv('./data/csv/test_acc10_300.csv')
model = load_model(model_path)

In [3]:
df.head()

,lig_id,pro_id,dest,score
0,2701,2701,./processed_data/test_acc10/2701_pro_2701_lig.npy,1
1,2701,2702,./processed_data/test_acc10/2702_pro_2701_lig.npy,0
2,2701,2703,./processed_data/test_acc10/2703_pro_2701_lig.npy,0
3,2701,2704,./processed_data/test_acc10/2704_pro_2701_lig.npy,0
4,2701,2705,./processed_data/test_acc10/2705_pro_2701_lig.npy,0


# WARNING! RESULTS SEEM CRAZY

In [5]:
# group by ligand
# for each group, compute top10 accuracy manually

matches = 0
total = float(len(df.lig_id.unique()))
debug = False

for lig_id, grp in tqdm(df.groupby('lig_id')):
    grp.reset_index(inplace=True)
    
    match_id = grp.index[grp['pro_id']==lig_id]
    
    # load all features
    probabilities = []
    n_channels = 2
    dims = (24,24,24)
    batch_size = 200
    
    for i in range(0, len(grp.index), batch_size):
        X = np.empty((batch_size, *dims, n_channels))
        sub_grp = grp[i*batch_size : i*batch_size+batch_size]
        for row in sub_grp.itertuples():
            X[row[0],] = np.load(row.dest)
        
        partial_probs = model.predict_on_batch(X)
        partial_probs = partial_probs.flatten()
        probabilities.extend(partial_probs)
    
    largest_first = reversed(np.argsort(probabilities).tolist())
    top10 = list(largest_first)[:10]
    
    if debug:
        print('ligid: ', lig_id)
        print('match_id: ', match_id)
        print('top_10: ', top10)
        print()
    
    if match_id in top10:
        matches += 1



100%|██████████| 300/300 [06:51<00:00,  1.36s/it]


0.6366666666666667


In [6]:
print(matches/total)    

0.6366666666666667
